Open and view H5 file

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import sys
# print(sys.executable)
# print(np.__version__)
# import tensorflow as tf
# print(tf.__file__)

In [ ]:
file_path = r"C:\Users\19jwp7\Documents\LF ML\Lane-Following-ML-main\QCarData.h5"

with h5py.File(file_path, 'r') as hf:
    # print("Datasets in the file:")
    # for dataset_name in hf:
    #     print(f" - {dataset_name}")

    images = hf["Binary_Image"][:]
    throttle = hf["Throttle"][:]
    steering = hf["Steering"][:]

    newImages = []
    for image in images:
        expanded = np.expand_dims(image, axis = -1)
        newImages.append(expanded)
        
    newImages = np.array(newImages)
    
    throttle = np.array(throttle)
    steering = np.array(steering)
    
    labels = np.column_stack((throttle, steering))

    # print("\nImages dataset (shape):", images.shape)
    # print("\nThrottle dataset (shape):", throttle.shape)
    # print("\nSteering dataset (shape):", steering.shape)
    
    # plt.imshow(newImages[0])

Machine Learning Model Below

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(newImages, labels, test_size=0.2, random_state=42)

In [ ]:
model = Sequential([
    # Convolutional layers to process images
    Conv2D(32, (3, 3), activation='relu', input_shape = (224, 224, 1)),  # Use your image shape here
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    
    # Fully connected layers for decision making
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(2)  # Outputs: throttle and steering
])

model.compile(optimizer='adam', loss=MeanSquaredError())

In [ ]:
# history = model.fit(newImages, labels, epochs=10, batch_size=32, validation_split=0.2)
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

In [ ]:
# model.save("LFML.h5")
model.save("LFML.keras")

In [ ]:

# model = load_model("LFML.h5", custom_objects={'MeanSquaredError': MeanSquaredError})
model = load_model("LFML.keras")